In [129]:
import pandas as pd

In [130]:
df_modern = pd.read_csv('raw/modern.csv')
df_modern.head()

,Fall,Term Code,College,Department,Degree,Major Code,Major Name,Concentration Code,Concentration Name,Total,...,Multiracial,International,Unknown.1,All African American,All Native American,All Hawaiian/ Pac Isl,All Asian,Illinois,Non-Illinois,Academic Program Code
0,2018,120188,KP,227,BS,5163,Agricultural & Biological Engr,,,52,...,1.0,7,1,1.0,0.0,0.0,7.0,37,15,10KP5163BS
1,2018,120188,KP,227,BS,5163,Agricultural & Biological Engr,5270,Agricultural Engineering,11,...,0.0,0,0,1.0,0.0,0.0,1.0,11,0,10KP5163BS
2,2018,120188,KP,227,BS,5163,Agricultural & Biological Engr,5271,Biological Engineering,17,...,3.0,4,0,2.0,0.0,0.0,5.0,10,7,10KP5163BS
3,2018,120188,KP,227,MENGR,1211,Engineering,5370,Energy Systems,30,...,0.0,17,0,0.0,0.0,0.0,1.0,9,21,10KS5370MENG
4,2018,120188,KP,227,MS,5163,Agricultural & Biological Engr,,,25,...,0.0,15,0,0.0,0.0,0.0,0.0,7,18,10KS5163MS


In [136]:
from collections import defaultdict

def processFile(file, year):
    readingHeader = True
    college = ""
    records = []
    
    for line in open(file):
        if "15 AGR, CONSUMER, & ENV SCIENCES" in line: readingHeader = False
            
        if readingHeader == True: continue
        line = line.strip()
        
        # Check if college:
        collegeCode = line[0:2].strip()
        if (collegeCode.isnumeric() and line[2] == ' ' and not line[3:5].isnumeric()):
            college = line[3:].strip()
            continue
        
        # Check if major:
        curriculumCode = line[0:6].strip()
        if not curriculumCode.isnumeric(): continue
            
        d = {}
        
        d['Fall'] = year
        d['College'] = college
        d['Legacy Major Name'] = line[7:22].strip()

        d['Legacy Department Code'] = line[23:27].strip()
        d['Legacy Advising Code'] = line[28:32].strip()
        
        d['Total'] = line[33:44].strip()
              
        remaining = [line[i:i+8].strip() for i in range(44, len(line), 8)]
        while len(remaining) < 11: remaining.append(0)
        
        #print(curriculumName)
        #print(line)
        #print(remaining)
        
        d['Male'], d['Female'], d['Unknown'], \
        d['Native American'], d['African American'], d['Asian American'], d['Hispanic'], d['Caucasian'], \
        d['International'], d['Non-Illinois'], d['Illinois'] = remaining
        
        records.append(d)
        
    return records

In [137]:
from os import listdir
from os.path import isfile, join

records = []
for file in [f for f in listdir('raw') if isfile(join('raw', f))]:
    if (file[-3:] != 'txt'): continue
    
    path = join('raw', file)
    year = file[2:6]
        
    print(path)
    records += processFile(path, year)
    
df_legacy = pd.DataFrame(records)
df_legacy.head()

raw\fa1980.txt
raw\fa1981.txt
raw\fa1982.txt
raw\fa1983.txt
raw\fa1984.txt
raw\fa1985.txt
raw\fa1986.txt
raw\fa1987.txt
raw\fa1988.txt
raw\fa1989.txt
raw\fa1990.txt
raw\fa1991.txt
raw\fa1992.txt
raw\fa1993.txt
raw\fa1994.txt
raw\fa1995.txt
raw\fa1996.txt
raw\fa1997.txt
raw\fa1998.txt
raw\fa1999.txt
raw\fa2000.txt
raw\fa2001.txt
raw\fa2002.txt
raw\fa2003.txt


,African American,Asian American,Caucasian,College,Fall,Female,Hispanic,Illinois,International,Legacy Advising Code,Legacy Department Code,Legacy Major Name,Male,Native American,Non-Illinois,Total,Unknown
0,,,77,"AGR, CONSUMER, & ENV SCIENCES",1980,47,,77,,1545,1545,AGR COMM,31,1,1,78,
1,,,1,"AGR, CONSUMER, & ENV SCIENCES",1980,1,,1,,1545,1545,AGR COMM,,,,1,
2,3,2,236,"AGR, CONSUMER, & ENV SCIENCES",1980,26,,237,,1505,1505,AGR ECON,216,1,5,242,
3,,,1,"AGR, CONSUMER, & ENV SCIENCES",1980,,,1,,1505,1505,AGR ECON,1,,,1,
4,,,70,"AGR, CONSUMER, & ENV SCIENCES",1980,13,,70,,1505,1505,AGR IND,57,,,70,


In [138]:
df = pd.concat([df_modern, df_legacy], ignore_index=True, sort=False)
df.head()

,Fall,Term Code,College,Department,Degree,Major Code,Major Name,Concentration Code,Concentration Name,Total,...,All African American,All Native American,All Hawaiian/ Pac Isl,All Asian,Illinois,Non-Illinois,Academic Program Code,Legacy Advising Code,Legacy Department Code,Legacy Major Name
0,2018,120188.0,KP,227.0,BS,5163.0,Agricultural & Biological Engr,,,52,...,1.0,0.0,0.0,7.0,37,15,10KP5163BS,NaN,NaN,NaN
1,2018,120188.0,KP,227.0,BS,5163.0,Agricultural & Biological Engr,5270,Agricultural Engineering,11,...,1.0,0.0,0.0,1.0,11,0,10KP5163BS,NaN,NaN,NaN
2,2018,120188.0,KP,227.0,BS,5163.0,Agricultural & Biological Engr,5271,Biological Engineering,17,...,2.0,0.0,0.0,5.0,10,7,10KP5163BS,NaN,NaN,NaN
3,2018,120188.0,KP,227.0,MENGR,1211.0,Engineering,5370,Energy Systems,30,...,0.0,0.0,0.0,1.0,9,21,10KS5370MENG,NaN,NaN,NaN
4,2018,120188.0,KP,227.0,MS,5163.0,Agricultural & Biological Engr,,,25,...,0.0,0.0,0.0,0.0,7,18,10KS5163MS,NaN,NaN,NaN


In [139]:
df["College"].unique()

array(['KP ', 'KM ', 'KT ', 'KV ', 'LC ', 'KY ', 'KL ', 'KN ', 'KR ',
       'KS ', 'LG ', 'NB ', 'KW ', 'LN ', 'LL ', 'KU ', 'LT ', 'LP ',
       'LE ', 'LM ', 'AGR, CONSUMER, & ENV SCIENCES',
       'COMMERCE & BUSINESS ADMIN', 'EDUCATION', 'ENGINEERING',
       'FINE & APPLIED ARTS', 'COLLEGE OF COMMUNICATIONS', 'LAW',
       'LIBERAL ARTS & SCIENCES', 'APPLIED LIFE STUDIES',
       'VETERINARY MEDICINE', 'INSTITUTE OF AVIATION',
       'LABOR & INDUSTRIAL RELATIONS', 'SCHOOL OF SOCIAL WORK',
       'LIBRARY & INFORMATION SCIENCE', 'COLLEGE OF BUSINESS',
       'GRADUATE COLLEGE', 'OFFICE OF CONTINUING EDUCATION'], dtype=object)

In [140]:
df = df.replace({
    'AGR, CONSUMER, & ENV SCIENCES': 'ACES',
    'EDUCATION': 'Education',
    'ENGINEERING': 'Engineering',
    'FINE & APPLIED ARTS': 'Fine and Applied Arts',
    'COLLEGE OF COMMUNICATIONS': 'Media',
    'LAW': 'Law',
    'LIBERAL ARTS & SCIENCES': 'LAS',
    'APPLIED LIFE STUDIES': 'Applied Life Sciences',
    'VETERINARY MEDICINE': 'VetMed',
    'INSTITUTE OF AVIATION': 'Aviation',
    'LABOR & INDUSTRIAL RELATIONS': 'Labor and Industrial Relations',
    'SCHOOL OF SOCIAL WORK': 'Social Work',
    'LIBRARY & INFORMATION SCIENCE': 'iSchool',
    'GRADUATE COLLEGE': 'Graduate',
    'COLLEGE OF BUSINESS': 'Business',
    'OFFICE OF CONTINUING EDUCATION': 'Continuing Education'
})
df.head()

,Fall,Term Code,College,Department,Degree,Major Code,Major Name,Concentration Code,Concentration Name,Total,...,All African American,All Native American,All Hawaiian/ Pac Isl,All Asian,Illinois,Non-Illinois,Academic Program Code,Legacy Advising Code,Legacy Department Code,Legacy Major Name
0,2018,120188.0,KP,227.0,BS,5163.0,Agricultural & Biological Engr,,,52,...,1.0,0.0,0.0,7.0,37,15,10KP5163BS,NaN,NaN,NaN
1,2018,120188.0,KP,227.0,BS,5163.0,Agricultural & Biological Engr,5270,Agricultural Engineering,11,...,1.0,0.0,0.0,1.0,11,0,10KP5163BS,NaN,NaN,NaN
2,2018,120188.0,KP,227.0,BS,5163.0,Agricultural & Biological Engr,5271,Biological Engineering,17,...,2.0,0.0,0.0,5.0,10,7,10KP5163BS,NaN,NaN,NaN
3,2018,120188.0,KP,227.0,MENGR,1211.0,Engineering,5370,Energy Systems,30,...,0.0,0.0,0.0,1.0,9,21,10KS5370MENG,NaN,NaN,NaN
4,2018,120188.0,KP,227.0,MS,5163.0,Agricultural & Biological Engr,,,25,...,0.0,0.0,0.0,0.0,7,18,10KS5163MS,NaN,NaN,NaN


In [142]:
df.to_csv('combined.csv')